# In Class Examples | Week 15 | AIST 2110 | Dataclasses

Housekeeping:
- Final Date: 12/8, 11am-1pm
    - Structure: 1hr for practical, 1hr for multiple choice

- Bonus Assignment


## Review

In [ ]:
def letter_count(s: str) -> dict[str, int]:
    counts: dict[str, int] = {}
    for ch in s:
        counts[ch] = counts.get(ch, 0) + 1
    return counts

print(letter_count("banana"))  # {'b':1, 'a':3, 'n':2}

## Dataclasses Basics

### Reg class ex

In [ ]:
# reg class example: Student without dataclasses

class Student:
    def __init__(self, name, id_number, gpa):
        self.name = name
        self.id_number = id_number
        self.gpa = gpa

    def is_honors(self):
        """Return True if the student has a high GPA."""
        return self.gpa >= 3.5

    def __repr__(self): # str for printing / debugging
        return f"Student(name={self.name!r}, id_number={self.id_number!r}, gpa={self.gpa!r})"


# Try it out:
s1 = Student("Alice", "J001", 3.8)
s2 = Student("Bob", "J002", 3.1)

print("s1:", s1)
print("s2:", s2)
print("Is s1 honors?", s1.is_honors())
print("Is s2 honors?", s2.is_honors())


### Dataclass ex

In [ ]:
from dataclasses import dataclass

@dataclass
class StudentDC:
    name: str
    id_number: str
    gpa: float

    def is_honors(self) -> bool:
        return self.gpa >= 3.5


# no __init__ and no __repr__ manually defined

s1 = StudentDC("Alice", "J001", 3.8)
s2 = StudentDC("Bob", "J002", 3.1)

print("s1:", s1)
print("s2:", s2)
print("Is s1 honors?", s1.is_honors())
print("Is s2 honors?", s2.is_honors())

Optional: we can get equality (==) and ordering (<, >, etc.) if needed
By default, dataclasses:

Compare by content, not by object identity, when you do ==

Are mutable (we can change attributes)

In [ ]:
s3 = StudentDC("Alice", "J001", 3.8)
s4 = StudentDC("Alice", "J001", 3.8)

print("s3:", s3)
print("s4:", s4)
print("s3 is s4?", s3 is s4)   # same object?
print("s3 == s4?", s3 == s4)   # same content?

### Default values & field factory

In [ ]:
from dataclasses import dataclass, field
from typing import List


@dataclass
class Course:
    code: str
    title: str
    credit_hours: int = 3 # if not provided, credit_hours = 3
    students: List[StudentDC] = field(default_factory=list)  # each Course gets its own list

    def enroll(self, student: StudentDC) -> None:
        self.students.append(student)

    def average_gpa(self) -> float:
        if not self.students:
            return 0.0
        total = sum(s.gpa for s in self.students)
        return total / len(self.students)
    
c1 = Course(code="AIST-2110", title="Intro to Python")
print(c1)  # students list starts empty

c1.enroll(StudentDC("Alice", "J001", 3.8))
c1.enroll(StudentDC("Bob", "J002", 3.1))
c1.enroll(StudentDC("Charlie", "J003", 3.5))

print("After enrollment:", c1)
print("Average GPA:", c1.average_gpa())


- Plain default: `students: list = []` is **dangerous** because the same list would be shared by all instances.
- Using `field(default_factory=list)` creates a **new list** for each instance.

## cool Dataclass Features

1. `frozen=True` to make instances immutable
2. `order=True` to get sorting on objects
3. `__post_init__` for validation and derived fields
4. Utility functions: `asdict`, `astuple`, `replace` (brief intro)

In [ ]:
# frozen == immutable
from dataclasses import dataclass


@dataclass(frozen=True)
class Point:
    x: float
    y: float


p = Point(2.0, 3.0)
print("Point:", p)

# Try to mutate (should raise dataclasses.FrozenInstanceError)
try:
    p.x = 10.0  # type: ignore[attr-defined]
except Exception as e:
    print("Error when trying to modify frozen dataclass:", e)

In [ ]:
# ordered dataclasses: enable sorting and comparisons

from dataclasses import dataclass


@dataclass(order=True)
class LeaderboardEntry:
    # When order=True, the fields are used in order to compare objects.
    score: int
    player_name: str


entries = [
    LeaderboardEntry(score=120, player_name="Alice"),
    LeaderboardEntry(score=95, player_name="Bob"),
    LeaderboardEntry(score=150, player_name="Charlie"),
]

print("Original list:")
print(entries)

print("\nSorted ascending (by score, then name):")
for e in sorted(entries):
    print(e)

print("\nSorted descending (highest score first):")
for e in sorted(entries, reverse=True):
    print(e)

In [ ]:
# __post_init__: validation and derived fields

from dataclasses import dataclass, field


@dataclass
class AssignmentGrade:
    name: str
    points_earned: float
    points_possible: float
    percentage: float = field(init=False)  # computed later

    def __post_init__(self):
        if self.points_possible <= 0:
            raise ValueError("points_possible must be > 0")

        self.percentage = (self.points_earned / self.points_possible) * 100

    def is_passing(self) -> bool:
        return self.percentage >= 70.0


hw1 = AssignmentGrade("HW1", points_earned=18, points_possible=20)
print(hw1)
print("Is passing?", hw1.is_passing())
print("Percentage:", hw1.percentage)

In [ ]:
# utility functions: asdict, astuple, replace

from dataclasses import asdict, astuple, replace


s = StudentDC("Dana", "J010", 3.9)

print("Original object:", s)
print("asdict:", asdict(s))
print("astuple:", astuple(s))

# replace returns a NEW instance with some fields changed
s_new = replace(s, gpa=4.0)
print("Replaced object:", s_new)
print("Original unchanged:", s)

## In class ex

In [ ]:
from dataclasses import dataclass


@dataclass
class Player:
    name: str
    level: int = 1
    experience: int = 0

    def xp_to_next_level(self) -> int:
        """
        Simple rule: XP needed = level * 100
        Level 1 -> need 100 XP
        Level 2 -> need 200 XP
        etc.
        """
        return self.level * 100

    def gain_xp(self, amount: int) -> None:
        """
        TODO:
        - Add `amount` to self.experience
        - While experience >= xp_to_next_level():
            * subtract xp_to_next_level() from experience
            * increase level by 1
            * (Optional) print a message that they leveled up!
        """
        pass


# Quick manual test:
p = Player("Hero")
print(p)
p.gain_xp(250)  # If implemented, should at least reach level 2
print(p)


### My version

In [ ]:
from dataclasses import dataclass


@dataclass
class Player:
    name: str
    level: int = 1
    experience: int = 0

    def xp_to_next_level(self) -> int:
        """
        Simple rule: XP needed = level * 100
        Level 1 -> need 100 XP
        Level 2 -> need 200 XP
        etc.
        """
        return self.level * 100

    def gain_xp(self, amount: int) -> None:
        """
        - Add `amount` to self.experience
        - While experience >= xp_to_next_level():
            * subtract xp_to_next_level() from experience
            * increase level by 1
            * (Optional) print a message that they leveled up!
        """
        self.experience += amount
        while self.experience >= self.xp_to_next_level():
            self.experience -= self.xp_to_next_level()
            self.level += 1
            print(f'{self.name} has leveled up to {self.level}!')


# Quick manual test:
p = Player("Hero")
print(p)
p.gain_xp(250)  # If implemented, should at least reach level 2
print(p)


## Harder ex

In [ ]:
from dataclasses import dataclass, field
from typing import List


@dataclass
class AssignmentGrade2:
    name: str
    points_earned: float
    points_possible: float
    percentage: float = field(init=False)

    def __post_init__(self):
        if self.points_possible <= 0:
            raise ValueError("points_possible must be > 0")
        self.percentage = (self.points_earned / self.points_possible) * 100

    def is_passing(self) -> bool:
        return self.percentage >= 70.0


@dataclass
class StudentRecord:
    student_name: str
    assignments: List[AssignmentGrade2] = field(default_factory=list)

    def add_assignment(self, assignment: AssignmentGrade2) -> None:
        """Add a new assignment grade."""
        self.assignments.append(assignment)

    def average_percentage(self) -> float:
        """
        TODO:
        - Return the average percentage across all assignments.
        - If there are no assignments, return 0.0
        """
        return 0.0

    def failing_assignments(self) -> List[AssignmentGrade2]:
        """
        TODO:
        - Return a list of assignments where is_passing() is False.
        """
        return []


# Example usage (once methods are implemented):
record = StudentRecord("Alice")

record.add_assignment(AssignmentGrade2("HW1", 18, 20))   # 90%
record.add_assignment(AssignmentGrade2("Quiz1", 7, 10))  # 70%
record.add_assignment(AssignmentGrade2("HW2", 10, 20))   # 50%

print(record)
print("Average %:", record.average_percentage())
print("Failing:", record.failing_assignments())


### My version

In [ ]:
from dataclasses import dataclass, field
from typing import List


@dataclass
class AssignmentGrade2:
    name: str
    points_earned: float
    points_possible: float
    percentage: float = field(init=False)

    def __post_init__(self):
        if self.points_possible <= 0:
            raise ValueError("points_possible must be > 0")
        self.percentage = (self.points_earned / self.points_possible) * 100

    def is_passing(self) -> bool:
        return self.percentage >= 70.0


@dataclass
class StudentRecord:
    student_name: str
    assignments: List[AssignmentGrade2] = field(default_factory=list)

    def add_assignment(self, assignment: AssignmentGrade2) -> None:
        """Add a new assignment grade."""
        self.assignments.append(assignment)

    def average_percentage(self) -> float:
        """
        - Return the average percentage across all assignments.
        - If there are no assignments, return 0.0
        """
        return 0.0

    def failing_assignments(self) -> List[AssignmentGrade2]:
        """
        - Return a list of assignments where is_passing() is False.
        """
        return []


# Example usage (once methods are implemented):
record = StudentRecord("Alice")

record.add_assignment(AssignmentGrade2("HW1", 18, 20))   # 90%
record.add_assignment(AssignmentGrade2("Quiz1", 7, 10))  # 70%
record.add_assignment(AssignmentGrade2("HW2", 10, 20))   # 50%

print(record)
print("Average %:", record.average_percentage())
print("Failing:", record.failing_assignments())


Created by Seth Barrett | 2025